# [LAB 06] 지도학습 > 예측 > 선형 > 07-선형회귀 결과보고
## #01.준비 작업
### [1] 패키지 참조

In [31]:
from hossam import *
from pandas import DataFrame, read_excel
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np
import statsmodels as sm

from itertools import combinations

#데이터 표준화 모듈
from sklearn.preprocessing import StandardScaler

#선형 회귀 분석 모듈
from sklearn.linear_model import LinearRegression

#훈련/검증 데이터 분리 모듈
from sklearn.model_selection import KFold,cross_val_predict,learning_curve,train_test_split,GridSearchCV

from scipy.stats import pearsonr,spearmanr


#성능 평가 지표 모듈
from sklearn.metrics import(
  r2_score,
  mean_absolute_error,
  mean_squared_error,
  mean_absolute_percentage_error
)

from scipy.stats import t,f
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson

import statsmodels.api as sm


### [2] 데이터 가져오기 + 데이터 분할
- statsmodels 패키지와 동일한 결과를 얻기 위해 훈련/검증 데이터 분할을 수행하지 않았다

In [32]:
origin = load_data('fish_processed')

yname = '무게'
x_train = origin.drop (columns=[yname])
y_train = origin[yname]

display(origin.head())

농어의 길이,높이,두께,무게를 조사한 데이터의 전처리 버전


,길이,높이,두께,무게
0,-2.180225,-2.016507,-1.896175,1.931521
1,-1.587434,-1.518703,-1.560774,3.496508
2,-1.442032,-1.417039,-1.316328,3.713572
3,-1.307815,-1.147103,-1.202633,3.960813
4,-1.173599,-1.147103,-1.026405,4.262680


### [3] 회귀 모델 구축

In [33]:
estimator = LinearRegression(n_jobs=-1)
estimator.fit(x_train,y_train)
estimator

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",-1
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


### [4] 변수 이름 구하기

In [34]:
yname = '무게'
xnames = list(origin.drop(yname,axis=1).columns)
yname,xnames

('무게', ['길이', '높이', '두께'])

## #03. 모형 적합도 보고
### (1) 잔차 구하기

In [35]:
y_train_pred = estimator.predict(x_train)

#잔차 계산
resid = y_train - y_train_pred  #실제값 - 예측치
resid[:5]

0   -1.307725
1   -0.325159
2   -0.263963
3   -0.172176
4    0.002476
Name: 무게, dtype: float64

### (2) durbin_watson 구하기 (독립성)

In [36]:
dw = durbin_watson(resid)
dw

np.float64(0.5639828667071892)

### (3) 설명력 R2
- 0~1 사이의 값
- 이 모형이 y 를 얼마나 잘 설명할 수 있는지를 나타냄

In [37]:
r2 =r2_score(y_train,y_train_pred)

r=np.sqrt(r2)
adj_r2 = 1-(1-r2)*(len(y_train)-1) / (len(y_train)-len(xnames)-1)
r2,adj_r2

(0.9485313491284232, 0.9455620038858322)

### (4) F 통계량과 p value
- 이 회귀모형 자체가 유용한가를 판단
- F 통계량은 전체 회귀모형의 유의성을 검정하는 지표
- F 가 크고, p value 가 작을수록 모형 전체가 유의해짐

$$
F = \frac{R^2 / p}{(1 - R^2) / (n - p - 1)}
$$

- p value 는 F 분포의 누적분포함수 (CDF) 를 이용해 계산하며, P<0.05 이면 모형이 통계적으로 유의함

- 귀무가설($H_0$): 모든 회귀계수는 0이다 (모형에 설명력이 없다)
- 대립가설($H_1$): 적어도 하나의 회귀계수는 0이 아니다


```python
f_statistic = (r2 / featurecount) / ((1 - r2) / (rowcount - featurecount - 1))

p = 1 - f.cdf(f_statistic, featurecount, rowcount - featurecount - 1)

- r2 : 결정계수 ($R^2$)
- featurecount : 독립변수 개수 ($p$)
- rowcount : 표본 수 ($n$)

```
- 의미 : 회귀 모형이 설명한 분산 / 설명하지 못한분산의 비율
  (F 값은 단순히 숫자가 아니라, 확률 변수로, F 분포를 따르는 확률 변수의 관측값 이라고 이해해야함)

  
- F 통계량이 클수록 모형의 설명력이 커짐

In [38]:
#표본수
rowcount = len(x_train)

#독립변수의 수
featurecount = len(xnames)

#F-Statistic
#statsmodels 와 내부 처리 방식에 의해 소수점 둘쨰자리부터 차이 발생 가능
#(통계적으로 무시할 수 있는 수준)
f_statistic = (r2 / featurecount) / ((1-r2) / (rowcount-featurecount-1))


#Prob F-statistic
p=1-f.cdf(f_statistic,featurecount,rowcount-featurecount-1)

print(f'F-statistic:{f_statistic}')
print(f'p-value:{p}')

F-statistic:319.44124769431
p-value:1.1102230246251565e-16


### (5) 모형 적합도 표

In [39]:
rdf = DataFrame(
    {
       'R':[r],
       'R2':[r2],
       'Adj R2' :[adj_r2],
       'F':[f_statistic],
       'p-value':[p],
       'Durbin-Watson':[dw] 

    }

)

rdf

,R,R2,Adj R2,F,p-value,Durbin-Watson
0,0.973926,0.948531,0.945562,319.441248,1.110223e-16,0.563983


### (6) 결과보고 문장

In [40]:
tpl = "R(%.3f), R²(%.3f), Adj R²(%.3f), F(%.3f), P-value(%.3f), Durbin-Watson(%.3f)"
tpl % (r, r2, adj_r2, f_statistic, p, dw)


'R(0.974), R²(0.949), Adj R²(0.946), F(319.441), P-value(0.000), Durbin-Watson(0.564)'

In [41]:
tpl = (
    "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, "
    "이 회귀모형은 통계적으로 %s "
    "(F(%s, %s) = %.3f, p %s 0.05)."
)

tpl % (
    yname,
    ", ".join(xnames),
    "유의하다" if p <= 0.05 else "유의하지 않다",
    len(x_train.columns),
    len(x_train.index) - len(x_train.columns) - 1,
    f_statistic,
    "<=" if p <= 0.05 else ">"
)


'무게에 대하여 길이, 높이, 두께로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 유의하다 (F(3, 52) = 319.441, p <= 0.05).'

## #04. 독립변수 통계량
#### (1) 절편의 계수를 하나의 배열로 결합

In [42]:
params = np.append(estimator.intercept_,estimator.coef_)
params

array([5.46864075, 0.82149299, 0.12690658, 0.0962184 ])

#### (2) 독립변수에 상수항 추가하기
- **상수항(절편)**은 \(y = \beta_0 + \beta_1 x_1 + \cdots + \beta_p x_p\)에서 \(\beta_0\)에 해당하며, 데이터의 평균적 위치(기준값)를 보정함  
- 상수항이 없으면 회귀선이 원점을 반드시 지나야 하므로, 실제 데이터 분포와 맞지 않을 수 있음  
- 상수항을 포함해야 잔차의 합이 0이 되고, 회귀계수의 추정이 불편성과 일치성을 가짐  

- **코드적 이유**
  - 행렬 연산(설계행렬 \(X\))에서 상수항을 추가해야 OLS 공식이 올바르게 적용됨  
  - `sm.add_constant()`로 상수항을 추가하면, 이후 행렬곱/역행렬/표준오차 등 모든 통계량 계산이 이론과 일치함  
  - 상수항이 없으면 표준오차, t-값, p-value 등 통계적 해석이 왜곡될 수 있음  

In [43]:
design_x = x_train.copy()
design_x = sm.add_constant(design_x)
design_x.head()

,const,길이,높이,두께
0,1.0,-2.180225,-2.016507,-1.896175
1,1.0,-1.587434,-1.518703,-1.560774
2,1.0,-1.442032,-1.417039,-1.316328
3,1.0,-1.307815,-1.147103,-1.202633
4,1.0,-1.173599,-1.147103,-1.026405


#### (3) 독립변수의 행렬곱

In [44]:
dot = np.dot(design_x.T,design_x)
dot

array([[ 5.60000000e+01, -1.04360964e-14,  8.21565038e-15,
        -8.88178420e-16],
       [-1.04360964e-14,  5.60000000e+01,  5.51947045e+01,
         5.45761176e+01],
       [ 8.21565038e-15,  5.51947045e+01,  5.60000000e+01,
         5.50391971e+01],
       [-8.88178420e-16,  5.45761176e+01,  5.50391971e+01,
         5.60000000e+01]])

#### (4) 행렬곱의 역행렬


In [45]:
inv = np.linalg.inv(dot)
inv

array([[ 1.78571429e-02,  1.96685433e-16, -2.45724118e-16,
         5.01069847e-17],
       [ 1.96685433e-16,  6.48339391e-01, -5.29174626e-01,
        -1.11758935e-01],
       [-2.45724118e-16, -5.29174626e-01,  9.56813327e-01,
        -4.24677510e-01],
       [ 5.01069847e-17, -1.11758935e-01, -4.24677510e-01,
         5.44165678e-01]])

#### (5) 역행렬의 대각선 반환

In [46]:
dia = inv.diagonal()
dia

array([0.01785714, 0.64833939, 0.95681333, 0.54416568])

#### (6) 평균 제곱오차 구하기

- **평균제곱오차(Mean Squared Error, MSE)**는 잔차(실제값-예측값)의 제곱합을 자유도 \((n - p)\)로 나눈 값  
- 여기서 \(n\)은 샘플 수, \(p\)는 회귀계수(상수항 포함) 개수  
- OLS(최소제곱법)에서 회귀계수의 분산 추정에 사용  

- **코드 설명**
  - `predictions = estimator.predict(x_train)` : 학습 데이터에 대한 예측값  
  - `MSE = ((y_train - predictions) ** 2).sum() / (n - p)` : 잔차 제곱합을 자유도로 나눔  
  - 자유도 계산을 명확히 하기 위해 `n = design_x.shape[0]`, `p = design_x.shape[1]`로 정의함  
  - sklearn의 API는 상수항이 고려되지 않은 MSE를 구한다(단순 모델 평가용)  
    - 상수항이 적용된 경우이므로 API를 통한 값이 아닌 직접 구한 값이 필요하다  


In [47]:
predictions = estimator.predict(x_train)

n = design_x.shape[0]  # 샘플 수
p = design_x.shape[1]  # 변수 개수(상수항 포함)

MSE = ((y_train - predictions) ** 2).sum() / (n - p)
MSE


np.float64(0.0633316530564705)

#### (7) 표준오차
- 회귀계수의 표준오차는 추정된 계수의 불확실성을나타냄
- 여기서 MSE 는 평균제곱오차, X 는 상수항이 포함된 설계 행렬
- 표준오차 = sqrt(MSE * diag((X'X)^(-1)))
- 역행렬의 대각원소와 MSE 를 곱해 표준오차 계산

In [48]:
se_b = np.sqrt(MSE*dia)
se_b

array([0.03362919, 0.20263367, 0.2461637 , 0.18564189])

#### (8) t_value 구하기
- t 값은 회귀계수의 통계적 유의성을 검정하는데 사용
- 자유도는 n-p(샘플 수 - 계수 개수)
- ts_b = params/se_b: 각 계수에 대해 표준오차로 나눠 t 값 계산

In [49]:
ts_b = params / se_b
ts_b

array([162.61589576,   4.05407935,   0.51553733,   0.5183011 ])

#### (9) p value 구하기
- p value 는 t-value 값이 관측된 값 이상이 될 확률로 계수의 유의성 판단에 사용
- 양측 검정 기준, p=2×(1−Ft​(∣t∣,df))
- 여기서 df 는 자유도 n-p
- 코드 설명 :
  t : 각 회귀계수의 t-value
  이렇게 구한 p-value가 0.05보다 작으면 해당 회귀계수는 통계적으로 유의하다고 판단한다.

In [50]:
n=design_x.shape[0]
n=design_x.shape[1]
p_values=[2*(1-t.cdf(np.abs(i),n-p)) for i in ts_b]
p_values

[np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan)]

#### (10) VIF 분산 팽창 지수 구하기
- VIF 는 다중공선성 진단 지표임
- VIF ≥ 10 → 다중공선성 문제를 강하게 의심
- 값이 클수록 해당 변수는 다른 변수들과 강한 상관관계를 가짐
- 각 독립변수에 대해 VIF 계산하여, VIF 값이 높으면 해당 변수 제거 또는 변환 고려

In [51]:
vif = []

for i,v in enumerate(xnames):
  j=list(x_train.columns).index(v)
  vif.append(variance_inflation_factor(x_train,j))

  print(vif)

[np.float64(36.30700589561463)]
[np.float64(36.30700589561463), np.float64(53.5815462967705)]
[np.float64(36.30700589561463), np.float64(53.5815462967705), np.float64(30.473277956534112)]


#### (11) 표준화 계수 (베타) 구하기
- 표준화 회귀 계수 B 는 변수 단위의 영향력 차이를 제거한 계수임
- 모든 변수(독립,종속) 을 표준정규분포 (평균 0 , 표준편차 1) 로 변환 후 회귀 분석
- B가 클수록 종속변수에 미치는 영향이 큼
- StandardScaler 로 전체 데이터 표준화
- 표준화된 데이터로 회귀 분석 후 coef_ 추출
- 즉, 독립 변수와 종속 변수를 표준화 한 후 머신러닝을 수행한 경우에 대한 계수값

In [52]:
scaler = StandardScaler()
std = scaler.fit_transform(origin)
std_df = DataFrame(std,columns=origin.columns)
std_df.head()

,길이,높이,두께,무게
0,-2.180225,-2.016507,-1.896175,-3.309048
1,-1.587434,-1.518703,-1.560774,-1.844971
2,-1.442032,-1.417039,-1.316328,-1.641903
3,-1.307815,-1.147103,-1.202633,-1.410604
4,-1.173599,-1.147103,-1.026405,-1.128201


In [53]:
scaler = StandardScaler()
std = scaler.fit_transform(origin)
std_df = DataFrame(std, columns=origin.columns)

std_x = std_df.drop(columns=[yname])
std_y = std_df[yname]

s_estimator = LinearRegression(n_jobs=-1)
s_estimator.fit(std_x, std_y)

beta = s_estimator.coef_
beta


array([0.76852356, 0.11872371, 0.09001429])

#### (12) 결과표 구성하기

In [57]:
result_df = DataFrame(
    {
        "종속변수": [yname] * len(xnames),
        "독립변수": xnames,
        "B(비표준계수)": np.round(params[1:], 4),
        "표준오차": np.round(se_b[1:], 4),
        "B(표준계수)": np.round(beta, 4),
        "t": np.round(ts_b[1:], 4),
        "유의확률": np.round(p_values[1:], 4),
        "VIF": np.round(vif, 4),
    }
)

# 유의확률에 따라 t별표 추가
result_df["t"] = result_df["t"].astype("str") + result_df["유의확률"].apply(
    lambda p: "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""
)

result_df


,종속변수,독립변수,B(비표준계수),표준오차,B(표준계수),t,유의확률,VIF
0,무게,길이,0.8215,0.2026,0.7685,4.0541,NaN,36.3070
1,무게,높이,0.1269,0.2462,0.1187,0.5155,NaN,53.5815
2,무게,두께,0.0962,0.1856,0.0900,0.5183,NaN,30.4733


#### (13) 독립변수 결과 보고 문자열

In [58]:
varstr = []

for n in xnames:
    item = result_df[result_df["독립변수"] == n]
    coef = float(item["B(비표준계수)"].values[0])
    pvalue = float(item["유의확률"].values[0])

    s = "%s가 증가하면 %s(이)가 %.3f만큼 변하는 것으로 나타남 (p %s 0.05, %s)" 
    k = s % (
        n,
        yname,
        coef,
        "<=" if pvalue <= 0.05 else ">",
        "유의함" if pvalue <= 0.05 else "유의하지 않음",
    )

    varstr.append(k)

varstr


['길이가 증가하면 무게(이)가 0.822만큼 변하는 것으로 나타남 (p > 0.05, 유의하지 않음)',
 '높이가 증가하면 무게(이)가 0.127만큼 변하는 것으로 나타남 (p > 0.05, 유의하지 않음)',
 '두께가 증가하면 무게(이)가 0.096만큼 변하는 것으로 나타남 (p > 0.05, 유의하지 않음)']